# Clustering IMDB movie actors

Defining notebook params:

In [1]:
seed = 42
n_spectral_components = 1000
# n_clusters = 25
min_cluster_size = 150
min_samples = 10
pdf_postfix = "_custom_1000"

Import stuff and ignore warnings:

In [2]:
import itertools
import pickle
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from IPython.core.display import display, HTML
from collections import Counter
from hdbscan import HDBSCAN
from matplotlib.axes._axes import _log
_log.setLevel("ERROR")
from matplotlib.patches import Patch
from networkx.algorithms.community import girvan_newman
from sklearn.cluster import SpectralClustering
from sklearn.manifold import spectral_embedding
from umap import UMAP

%matplotlib inline

Loading the network and taking the largest connected component:

In [3]:
np.random.seed(seed)

with open("data/edges.pkl", "rb") as f:
    G = nx.Graph()
    G.add_edges_from(pickle.load(f))

G.remove_nodes_from(set(nx.nodes(G)) - max(nx.connected_components(G), key=len))
adj_matrix = nx.to_numpy_array(G)
node_list = list(G.nodes())

print(f"nodes: {len(node_list)}")
print(f"edges: {len(G.edges())}")

nodes: 13406
edges: 390859


Using Spectral Embedding, reducing dimensions with UMAP and clustering with HDBSCAN:

In [4]:
np.random.seed(seed)
random_state = np.random.RandomState(seed)

embedding = spectral_embedding(
    adj_matrix, n_components=n_spectral_components, drop_first=False, random_state=random_state
)
embedding = UMAP(
    n_components=2, n_neighbors=30, min_dist=0.0, random_state=random_state
).fit_transform(embedding)

clusters: 20
cluster sizes: [(17, 3189), (-1, 1480), (18, 1048), (10, 757), (4, 722), (9, 713), (3, 641), (15, 620), (19, 603), (12, 555), (13, 445), (11, 422), (2, 406), (8, 378), (5, 296), (14, 242), (16, 229), (1, 178), (6, 177), (0, 154), (7, 151)]
noise level: 0.11


In [ ]:
np.random.seed(seed)
random_state = np.random.RandomState(seed)

clusters = HDBSCAN(
    min_samples=min_samples, min_cluster_size=min_cluster_size, core_dist_n_jobs=-1
).fit_predict(embedding)

# clusters = SpectralClustering(
#     n_clusters=n_clusters, affinity="precomputed", random_state=random_state, n_jobs=-1
# ).fit_predict(adj_matrix)

# iter = girvan_newman(G)
# communities = list(itertools.takewhile(lambda c: len(c) <= n_clusters, iter))[-1]
# clusters_dict = {node: i for i, c in enumerate(communities) for node in c}
# clusters = [clusters_dict[node] for node in node_list]

counter = Counter(clusters)
print(f"clusters: {np.amax(clusters) + 1}")
print(f"cluster sizes: {sorted(counter.items(), key=lambda x: x[1], reverse=True)}")
print(f"noise level: {np.round(counter[-1] / len(clusters), 3)}")

Defining some helper functions for printing leaderboards:

In [ ]:
def display_side_by_side(dfs, captions):
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

def print_leaderboards(df):
    df_list = []
    caption_list = []
    for i in range(-1, df["cluster"].max() + 1):
        caption_list.append(f"Cluster idx: {i}")
        cluster_top_actors = df_actor_info[df_actor_info["cluster"] == i].reset_index()
        cluster_top_actors = cluster_top_actors.drop(["cluster", "index", "nconst"], axis=1)
        df_list.append(cluster_top_actors.head())
    display_side_by_side(df_list, caption_list)    

d = {"nconst": node_list, "cluster": clusters}
df_actor_info = pd.DataFrame(d)
df_actor_info["degree"] = df_actor_info["nconst"].apply(G.degree)

betweenness = nx.betweenness_centrality(G)
df_betweenness = pd.DataFrame(betweenness.items(), columns=["nconst", "betweenness"])
df_actor_info = df_actor_info.merge(df_betweenness, how="inner")

df_names = pd.read_csv("data/actor_names_birthyear.csv", sep=",").reset_index()
df_names = df_names.drop(["index"], axis=1)
df_actor_info = df_actor_info.merge(df_names, how="left")

birth_years = df_actor_info.sort_values(by="nconst", ascending=True)["birthYear"].values
birth_years[birth_years == "\\N"] = np.nan
birth_years = birth_years.astype(float)

Printing leaderboards for each cluster when sorted by degree:

In [ ]:
df_actor_info = df_actor_info.sort_values(by="degree", ascending=False)
print_leaderboards(df_actor_info)

Printing leaderboards for each cluster when sorted by betweenness:

In [ ]:
df_actor_info = df_actor_info.sort_values(by="betweenness", ascending=False)
print_leaderboards(df_actor_info)

Visualizing the network using Spring and Custom layouts:

In [ ]:
np.random.seed(seed)

def draw_network(G, pos, patch_list, node_colors, pdf_name, node_size=1, edge_width=.1, font_size=1):
    plt.figure(figsize=(12, 9))
    nx.draw_networkx_nodes(G, pos=pos, node_size=node_size, node_color=node_colors)
    nx.draw_networkx_edges(G, pos=pos, width=edge_width)
    # nx.draw_networkx_labels(G, pos=pos, font_size=font_size)
    plt.legend(handles=patch_list, loc="center left", bbox_to_anchor=(1, 0.5), ncol=2, fancybox=True)
    plt.savefig(f"results/{pdf_name + pdf_postfix}.pdf", bbox_inches="tight")
    plt.show()

palette = plt.cm.Set3.colors \
          + tuple(plt.cm.tab20b.colors[i] for i in [0, 4, 8, 12, 16]) \
          + tuple(plt.cm.tab20c.colors[i] for i in [4]) \
          + plt.cm.tab10.colors
node_colors = [palette[cluster] for cluster in clusters]
patch_list = [Patch(color=palette[i], label=i) for i in range(-1, np.amax(clusters) + 1)]

pos_sp = nx.spring_layout(G, seed=seed)
pos_su = {node_list[i]: embedding[i] for i in range(len(node_list))}

draw_network(G, pos=pos_sp, patch_list=patch_list, node_colors=node_colors, pdf_name="graph_sp")
draw_network(G, pos=pos_su, patch_list=patch_list, node_colors=node_colors, pdf_name="graph_su")

Visualizing birth years using custom layout:

In [ ]:
plt.figure(figsize=(15, 9))
g = nx.draw_networkx_nodes(
    G, pos=pos_su, node_size=1, node_color=birth_years, cmap=plt.cm.gist_rainbow, vmin=np.nanmin(birth_years),
    vmax=np.nanmax(birth_years)
)
nx.draw_networkx_edges(G, pos=pos_su, width=.1)
plt.colorbar(g)
plt.savefig(f"results/birth_years{pdf_postfix}.pdf", bbox_inches="tight")
plt.show()